In [1]:
from prepare_language_folder import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import MultipleyeDataCollection
from pathlib import Path

/Users/anastassiashaitarova/miniforge3/envs/pymovements_user/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_collection_name = 'MultiplEYE_SQ_CH_Zurich_1_2025'

If necessary, prepare the data folder by unzipping the downloaded files. Works only for MultiplEYE and MeRID data collections so far. Also, there might be some manual steps necessary.

In [3]:
prepare_language_folder(data_collection_name)

No zipped data found for test_sessions. Proceeding with existing data.
No zipped data found for 013_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for .DS_Store. Proceeding with existing data.
No zipped data found for 006_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 012_SQ_CH_1_ET1_start_after_trial_5. Proceeding with existing data.
No zipped data found for 010_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 009_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for data_piloting_stimuli_MultiplEYE_SQ_CH_Zurich_1_2025participant_id_1_to_5. Proceeding with existing data.
No zipped data found for 007_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for NotFoundError. Proceeding with existing data.
No zipped data found for 012_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for pilot_sessions. Proceeding with existing data.
No zipped data found for 008_SQ_CH_1_ET1. Proceeding with

In [4]:
this_repo = Path().resolve()
data_folder_path = this_repo / "data" / data_collection_name

## Converting EDF to ASCII and Performing Data Collection for Multipleye

In [5]:
multipleye_sq = MultipleyeDataCollection.create_from_data_folder(data_folder_path)

Folder test_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.
Folder data_piloting_stimuli_MultiplEYE_SQ_CH_Zurich_1_2025participant_id_1_to_5 does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.
Folder pilot_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.


Converting EDF to ASC: 100%|██████████| 9/9 [00:00<00:00, 9965.35it/s]
Parsing participant data : 012_SQ_CH_1_ET1_start_after_trial_5:   0%|          | 0/9 [00:00<?, ?it/s]WARNING:root:Session 012_SQ_CH_1_ET1_start_after_trial_5 has been restarted.
Parsing participant data : 011_SQ_CH_1_ET1: 100%|██████████| 9/9 [00:00<00:00, 470.69it/s]


In [6]:
sessions = [s for s in multipleye_sq]
sessions[0]

{'session_folder_path': '/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/data/MultiplEYE_SQ_CH_Zurich_1_2025/eye-tracking-sessions/006_SQ_CH_1_ET1',
 'session_file_path': PosixPath('/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/data/MultiplEYE_SQ_CH_Zurich_1_2025/eye-tracking-sessions/006_SQ_CH_1_ET1/006sqch1.edf'),
 'session_file_name': '006sqch1.edf',
 'session_folder_name': '006_SQ_CH_1_ET1',
 'session_stimuli': '',
 'is_pilot': False,
 'asc_path': PosixPath('/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/data/MultiplEYE_SQ_CH_Zurich_1_2025/eye-tracking-sessions/006_SQ_CH_1_ET1/006sqch1.asc'),
 'gaze_path': PosixPath('/Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/data/MultiplEYE_SQ_CH_Zurich_1_2025/quality_reports/006_SQ_CH_1_ET1/006_SQ_CH_1_ET1_gaze.pkl'),
 'completed_stimuli_ids': [13, 7, 2, 8, 10, 6, 9, 12, 3, 4, 1, 11],
 'stimuli_tr

## Creating Gaze Frame from ASCII File

In [25]:
from pymovements.gaze.io import from_asc

raw_file = sessions[0]['asc_path']

gaze = from_asc(
    file=raw_file,
    events=True
)

# Set screen parameters for degrees of visual angle conversion
# a must-have in order to convert pixels to degrees of visual angle
gaze.experiment.screen.distance_cm = 60 
gaze.experiment.screen.height_cm = 28
gaze.experiment.screen.width_cm = 37

## Saving Gaze

In [26]:
# Save data from the Gaze object in the provided directory.

# def save(
#     self,
#     dirpath: str | Path,
#     *,
#     save_events: bool | None = None,
#     save_samples: bool | None = None,
#     save_experiment: bool | None = None,
#     verbose: int = 1,
#     extension: str = 'feather',
# ) -> Gaze:

#     Depending on parameters it may save three files:
#     * preprocessed gaze in samples (samples)
#     * calculated gaze events (events)
#     * metadatata experiment in YAML file (experiment).

#     Verbosity level (0: no print output, 1: show progress bar, 2: print saved filepaths)

#     Data will be saved as feather or csv files.
#     there is no built-in pickle save function in pymovements

gaze.save(Path(raw_file).parent, save_events=True,
    save_samples=True, save_experiment=True, extension='feather')
# more human-readable csv format
gaze.save(Path(raw_file).parent, save_events=True,
    save_samples=True, save_experiment=True, extension='csv', verbose=1)

time,pupil,pixel
i64,f64,list[f64]
2626911,996.0,"[50.7, 122.8]"
2626912,996.0,"[50.4, 124.0]"
2626913,998.0,"[50.8, 123.3]"
2626914,995.0,"[51.8, 123.0]"
2626915,995.0,"[51.6, 123.1]"
…,…,…
10558292,879.0,"[278.5, 885.7]"
10558293,880.0,"[278.7, 886.1]"
10558294,874.0,"[277.6, 885.6]"


## Calculating Fixations

### Compute gaze positions

In [27]:
# Compute gaze positions in degrees of visual angle from pixel position coordinates.
# This method requires a properly initialized :py:attr:`~.Gaze.experiment` attribute.
# After success, :py:attr:`~.Gaze.samples` is extended by the resulting dva position columns.

gaze.pix2deg()

### Compute gaze velocity

In [28]:
# Compute gaze velocity in dva/s from dva position coordinates.
# This method requires a properly initialized :py:attr:`~.Gaze.experiment` attribute.
# After success, :py:attr:`~.Gaze.samples` is extended by the resulting velocity columns.

# Parameters
# ----------
# method: str
#     Computation method. See :func:`~transforms.pos2vel()` for details, default: fivepoint.
#     (default: 'fivepoint')
# **kwargs: int | float | str
#     Additional keyword arguments to be passed to the :func:`~transforms.pos2vel()` method.

gaze.pos2vel('smooth')

In [29]:
gaze

time,pupil,pixel,position,velocity
i64,f64,list[f64],list[f64],list[f64]
2626911,996.0,"[50.7, 122.8]","[-15.865846, -9.976727]","[null, null]"
2626912,996.0,"[50.4, 124.0]","[-15.873338, -9.945663]","[null, null]"
2626913,998.0,"[50.8, 123.3]","[-15.863349, -9.963784]","[9.574565, -3.020216]"
2626914,995.0,"[51.8, 123.0]","[-15.838371, -9.97155]","[4.57903, 0.431618]"
2626915,995.0,"[51.6, 123.1]","[-15.843367, -9.968961]","[-12.486222, 10.786545]"
…,…,…,…,…
10558292,879.0,"[278.5, 885.7]","[-10.031679, 10.170738]","[-6.543484, -14.647334]"
10558293,880.0,"[278.7, 886.1]","[-10.026444, 10.181079]","[-4.36183, -12.924581]"
10558294,874.0,"[277.6, 885.6]","[-10.055232, 10.168153]","[5.235921, 4.738033]"


### Compute New Fixations

In [30]:
# Detect events by applying a specific event detection method.

# Parameters
# ----------
# method: Callable[..., pm.Events] | str
#     The event detection method to be applied.
# eye: str
#     Select which eye to choose. Valid options are ``auto``, ``left``, ``right`` or ``None``.
#     If ``auto`` is passed, eye is inferred in the order ``['right', 'left', 'eye']`` from
#     the available columns in :py:attr:`~.Gaze.samples`. (default: 'auto')
# clear: bool
#     If ``True``, event DataFrame will be overwritten with new DataFrame instead of being
#     merged into the existing one. (default: False)
# **kwargs: Any
#     Additional keyword arguments to be passed to the event detection method.

for th in [5, 10, 20, 30]:
    gaze.detect(method="ivt", velocity_threshold=th)
    print(th, gaze.events.frame["name"].unique())

5 shape: (3,)
Series: 'name' [str]
[
	"saccade_eyelink"
	"blink_eyelink"
	"fixation_eyelink"
]
10 shape: (3,)
Series: 'name' [str]
[
	"blink_eyelink"
	"saccade_eyelink"
	"fixation_eyelink"
]
20 shape: (4,)
Series: 'name' [str]
[
	"blink_eyelink"
	"fixation_eyelink"
	"saccade_eyelink"
	"fixation"
]
30 shape: (4,)
Series: 'name' [str]
[
	"fixation_eyelink"
	"blink_eyelink"
	"fixation"
	"saccade_eyelink"
]


In [31]:
gaze.detect(method="ivt", velocity_threshold=30, name="fixation.ivt")
gaze

time,pupil,pixel,position,velocity
i64,f64,list[f64],list[f64],list[f64]
2626911,996.0,"[50.7, 122.8]","[-15.865846, -9.976727]","[null, null]"
2626912,996.0,"[50.4, 124.0]","[-15.873338, -9.945663]","[null, null]"
2626913,998.0,"[50.8, 123.3]","[-15.863349, -9.963784]","[9.574565, -3.020216]"
2626914,995.0,"[51.8, 123.0]","[-15.838371, -9.97155]","[4.57903, 0.431618]"
2626915,995.0,"[51.6, 123.1]","[-15.843367, -9.968961]","[-12.486222, 10.786545]"
…,…,…,…,…
10558292,879.0,"[278.5, 885.7]","[-10.031679, 10.170738]","[-6.543484, -14.647334]"
10558293,880.0,"[278.7, 886.1]","[-10.026444, 10.181079]","[-4.36183, -12.924581]"
10558294,874.0,"[277.6, 885.6]","[-10.055232, 10.168153]","[5.235921, 4.738033]"


## Calculating Saccades

In [32]:
import polars as pl

gaze.detect('microsaccades', minimum_duration=12)

gaze.events.frame.filter(pl.col("name") == "microsaccades").head()

name,onset,offset,duration
str,i64,i64,i64


### Areas Of Interest

### Loading AOI File into DataFrame

In [33]:
from pymovements.stimulus.text import from_file

In [34]:
aoi_chars_file = "data/MultiplEYE_SQ_CH_Zurich_1_2025/eye-tracking-sessions/data_piloting_stimuli_MultiplEYE_SQ_CH_Zurich_1_2025participant_id_1_to_5/aoi_stimuli_sq_ch_1/lit_magicmountain_6_aoi.csv"

stimulus = from_file(
    aoi_path=aoi_chars_file,
    aoi_column="char",
    start_x_column="top_left_x",
    start_y_column="top_left_y",
    width_column="width",
    height_column="height",
    page_column="page",
)

stimulus.aois.head(10)

char_idx,char,top_left_x,top_left_y,width,height,char_idx_in_line,line_idx,page,word_idx,word_idx_in_line,word
i64,str,f64,f64,i64,i64,i64,i64,str,i64,i64,str
0,"""M""",81.0,89.0,14,33,0,0,"""page_1""",0,0,"""Mali"""
1,"""a""",95.0,89.0,14,33,1,0,"""page_1""",0,0,"""Mali"""
2,"""l""",109.0,89.0,14,33,2,0,"""page_1""",0,0,"""Mali"""
3,"""i""",123.0,89.0,14,33,3,0,"""page_1""",0,0,"""Mali"""
4,""" """,137.0,89.0,14,33,4,0,"""page_1""",0,0,""" """
5,"""M""",151.0,89.0,14,33,5,0,"""page_1""",1,1,"""Magjik"""
6,"""a""",165.0,89.0,14,33,6,0,"""page_1""",1,1,"""Magjik"""
7,"""g""",179.0,89.0,14,33,7,0,"""page_1""",1,1,"""Magjik"""
8,"""j""",193.0,89.0,14,33,8,0,"""page_1""",1,1,"""Magjik"""


### Mapping Fixations to AOI 

In [35]:
print(gaze.samples.columns)

['time', 'pupil', 'pixel', 'position', 'velocity']


In [36]:
#  We map each gaze point to an aoi, considering the boundary still part of the area of interest.

# Parameters
# ----------
# aoi_dataframe: pm.stimulus.TextStimulus
#     Area of interest dataframe.
# eye: str
#     String specificer for inferring eye components. Supported values are: auto, mono, left
#     right, cyclops. Default: auto.
# gaze_type: str
#     String specificer for whether to use position or pixel coordinates for
#     mapping. Default: pixel.


# explode the list column "pixel" into two numeric columns
gaze.samples = gaze.samples.with_columns([
    pl.col("pixel").list.get(0).alias("pixel_xr"),
    pl.col("pixel").list.get(1).alias("pixel_yr"),
])

gaze.map_to_aois(
    aoi_dataframe=stimulus,
    eye="auto",
    gaze_type="pixel"
)

5565it [00:01, 4387.20it/s]/Users/anastassiashaitarova/miniforge3/envs/pymovements_user/lib/python3.13/site-packages/pymovements/stimulus/text.py:277: UserWarning: Comparisons with None always result in null. Consider using `.is_null()` or `.is_not_null()`.
  (aoi_dataframe.aois[aoi_dataframe.start_x_column] <= row[x_eye]) &
5633it [00:01, 4363.34it/s]


TypeError: argument 'rhs': must be real number, not NoneType

In [37]:
print(stimulus.aois.null_count())


shape: (1, 12)
┌──────────┬──────┬────────────┬────────────┬───┬──────┬──────────┬──────────────────┬──────┐
│ char_idx ┆ char ┆ top_left_x ┆ top_left_y ┆ … ┆ page ┆ word_idx ┆ word_idx_in_line ┆ word │
│ ---      ┆ ---  ┆ ---        ┆ ---        ┆   ┆ ---  ┆ ---      ┆ ---              ┆ ---  │
│ u32      ┆ u32  ┆ u32        ┆ u32        ┆   ┆ u32  ┆ u32      ┆ u32              ┆ u32  │
╞══════════╪══════╪════════════╪════════════╪═══╪══════╪══════════╪══════════════════╪══════╡
│ 0        ┆ 0    ┆ 0          ┆ 0          ┆ … ┆ 0    ┆ 0        ┆ 0                ┆ 0    │
└──────────┴──────┴────────────┴────────────┴───┴──────┴──────────┴──────────────────┴──────┘


Step 1 pf peyepline: create the gaze frame.

	-- data collection folder
	---- ...
	---- fixations
	---- saccades(?)
	---- reading_measures
	---- raw_data (i.e. gaze sample csv)